In [13]:
import os
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib

Using matplotlib backend: Qt5Agg


### Goals:


In [29]:
BREADCRUMB_DIR = '/home/kyle/datasets/tktrips/raw-data/breadcrumbs/'
DATA_DIR = '/home/kyle/datasets/tktrips/raw-data/jan22-datapull/'
DATA_OUT = '/home/kyle/datasets/tktrips/deadheading/'

In [51]:
# concatenate the breadcrumb csvs into one DF, did sanity check for multiple header writes
all_files = [BREADCRUMB_DIR + f for f in os.listdir(BREADCRUMB_DIR)]
breadcrumbs = pd.concat((pd.read_csv(f, engine='python') for f in all_files))

In [58]:
#sort breadcrumbs by trip then timestamp
breadcrumbs.sort_values(['trip_id', 'timestamp'], ascending=[True, True], inplace=True)
breadcrumbs.reset_index(drop=True, inplace=True)

In [63]:
test_duplicate_latlong_idxs = []
def mark_dup_geocodes(row):
    """Input: row. Output: row index to mark for del"""
    idx = row.name
    if idx == 0:
        return
    
#     prev_row = test_bc.iloc[idx - 1] # uncomment to use for test df
    prev_row = breadcrumbs.iloc[idx - 1] #get previous row
    
    # Compare previous row's lat/long, if same mark new row for del
    if row[2] == prev_row[2] and row[3] == prev_row[3]: 
        duplicate_latlong_idxs.append(idx)
        return

In [64]:
%%time
# Run through rows and mark the idxs to delete
duplicate_latlong_idxs = []
breadcrumbs.apply(mark_dup_geocodes, axis=1)
print(len(duplicate_latlong_idxs))

550504
CPU times: user 2min 43s, sys: 8 ms, total: 2min 43s
Wall time: 2min 43s


In [65]:
# Sanity Check
# Make sure the list is unique w/ set
unique_duplicate_latlong_idxs = set(duplicate_latlong_idxs)
print(len(unique_duplicate_latlong_idxs))

550504


In [66]:
%%time
# Remove data rom the DF and reset the index
cleaned_breadcrumbs = breadcrumbs.drop(breadcrumbs.index[duplicate_latlong_idxs])
cleaned_breadcrumbs.reset_index(drop=True, inplace=True)
print(cleaned_breadcrumbs.shape)

(394436, 5)
CPU times: user 180 ms, sys: 20 ms, total: 200 ms
Wall time: 206 ms


In [67]:
#Sanity Check
if len(cleaned_breadcrumbs) + len(duplicate_latlong_idxs) == len(breadcrumbs):
    print("Boom!")
else:
    print("Damn!")

Boom!


In [69]:
%%time
# Save cleaned breadcrumbs to csv
cleaned_breadcrumbs.to_csv(DATA_OUT + "breadcrumbs.csv", index=False)

CPU times: user 1.65 s, sys: 52 ms, total: 1.7 s
Wall time: 1.72 s
